<a href="https://colab.research.google.com/github/suryamn1/deep-learning-projects/blob/main/vgg16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers import Dense 
from keras.layers import Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from PIL import Image
import numpy as np
import pandas as pd
import os
from os import listdir
from PIL import Image
from numpy import array
from numpy import asarray
import seaborn as sns

In [18]:
from google.colab import files
upload=files.upload()

In [7]:
import zipfile,os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input
"""
!wget --no-check-certificate \
    "https://github.com/giovannipcarvalho/banana-ripeness-classification/archive/refs/heads/master.zip" \
    -O "/tmp/bananas.zip"
"""

zip_ref = zipfile.ZipFile('/content/drive/MyDrive/banana_dataset.zip', 'r') #Opens the zip file in read mode
zip_ref.extractall('/bannana_tmp') #Extracts the files into the /tmp folder
zip_ref.close()
img_height,img_width=200,200
batch_size=32
images_dir = '/bannana_tmp/Fayoum_University_Banana _Classes'
shape_sum = 0
class_name_numb = dict()
train_avg_shape = 80
training_img_list = list()
for dirname, _, filenames in os.walk(images_dir):
    for filename in filenames:
        img_data = Image.open(os.path.join(dirname, filename))
        
        resizedImage = img_data.resize((train_avg_shape, train_avg_shape))
        resizedImage = resizedImage.convert('RGB')
        resizedImage = asarray(resizedImage)/255
        
        class_label = dirname.split('/')[-1]
        training_img_list.append([resizedImage, class_label])
        shape_sum += np.max(img_data.size)
        class_name_numb[class_label] = len(class_name_numb)-1

***Data Preprocessing***

In [8]:
import zipfile,os
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2) 

train_generator = train_datagen.flow_from_directory(
    images_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='training')

validation_generator = train_datagen.flow_from_directory(
    images_dir, 
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary',
    subset='validation') 

from sklearn.model_selection import KFold
from numpy import asarray
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix

np.random.seed(17)
kf = KFold(n_splits=5)
epochs = 20
batch_size = 32

Found 353 images belonging to 3 classes.
Found 87 images belonging to 3 classes.


In [9]:
from keras.applications.vgg16 import VGG16
vgg16_model = VGG16(weights='imagenet',include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [10]:
len(vgg16_model.layers)

19

In [11]:
vgg16_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [12]:
x=vgg16_model.output

In [13]:
from keras.layers import Dense,GlobalAveragePooling2D
from keras.layers import BatchNormalization
x=GlobalAveragePooling2D()(x)
x=BatchNormalization()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x) 
x=Dense(512,activation='relu')(x) 
preds=Dense(len(class_name_numb), activation='softmax')(x)

In [14]:
from keras.models import Model
newModel=Model(inputs=vgg16_model.input,outputs=preds)


In [15]:
for layer in newModel.layers[:-5]:
    layer.trainable=False

In [16]:
print(len(newModel.layers))
newModel.summary()

25
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0  

In [17]:
newModel.compile(optimizer='Adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size
history = newModel.fit_generator(
    train_generator,
    steps_per_epoch = step_size_train,
    validation_data = validation_generator, 
    validation_steps = validation_generator.samples // batch_size,
    epochs =20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  if __name__ == '__main__':


Epoch 1/20
11/11 [==============================] - 162s 15s/step - loss: 0.9512 - accuracy: 0.5888 - val_loss: 0.9843 - val_accuracy: 0.5156
Epoch 2/20
11/11 [==============================] - 159s 15s/step - loss: 0.3885 - accuracy: 0.8318 - val_loss: 0.9117 - val_accuracy: 0.6250
Epoch 3/20
11/11 [==============================] - 159s 15s/step - loss: 0.2284 - accuracy: 0.8972 - val_loss: 0.8590 - val_accuracy: 0.5000
Epoch 4/20
11/11 [==============================] - 159s 15s/step - loss: 0.1387 - accuracy: 0.9346 - val_loss: 0.9382 - val_accuracy: 0.6250
Epoch 5/20
11/11 [==============================] - 159s 15s/step - loss: 0.1479 - accuracy: 0.9439 - val_loss: 0.8968 - val_accuracy: 0.5938
Epoch 6/20
11/11 [==============================] - 159s 14s/step - loss: 0.1731 - accuracy: 0.9595 - val_loss: 0.7821 - val_accuracy: 0.6719
Epoch 7/20
11/11 [==============================] - 160s 15s/step - loss: 0.1125 - accuracy: 0.9595 - val_loss: 0.7620 - val_accuracy: 0.5469
Epoch 